# Chapter 9. Up and Running with TensorFlow

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

W0502 16:06:01.809484 140735606014848 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [4]:
with tf.Session() as sess: 
    x.initializer.run() 
    y.initializer.run() 
    result = f.eval()

In [5]:
result

42

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run() # actually initialize all the variables 
    result = f.eval()


In [7]:
result

42

#### Inside Jupyter or within a Python shell you may prefer to create an InteractiveSession. The only difference from a regular Session is that when an InteractiveSession is created it automatically sets itself as the default session, so you don’t need a with block (but you do need to close the session manually when you are done with it):

In [8]:
sess = tf.InteractiveSession()
init.run() 
result = f.eval()
sess.close()
result

42

## Managing Graphs

In [9]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [11]:
x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

#### In Jupyter (or in a Python shell), it is common to run the same commands more than once while you are experimenting. As a result, you may end up with a default graph containing many duplicate nodes. One solution is to restart the Jupyter kernel (or the Python shell), but a more convenient solution is to just reset the default graph by running tf.reset_default_graph().
 

## Lifecycle of a Node Value

In [13]:
w = tf.constant(3) 
x=w+2 
y=x+5 
z=x*3

In [14]:
with tf.Session() as sess: 
    print(y.eval()) # 10 
    print(z.eval()) # 15

10
15


#### if you want to evaluate y and z efficiently, without evaluating w and x twice as in the previous code, you must ask TensorFlow to evaluate both y and z in just one graph run, as shown in the following code:

In [15]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val) # 10 
    print(z_val) # 15

10
15


## Linear Regression with TensorFlow

In [28]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
theta_value

X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


### Implementing Gradient Descent

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

n_epochs = 1000 
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X") 
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") 
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 12.299408
Epoch 100 MSE = 0.8240509
Epoch 200 MSE = 0.61104304
Epoch 300 MSE = 0.5842089
Epoch 400 MSE = 0.5674804
Epoch 500 MSE = 0.55548537
Epoch 600 MSE = 0.5468306
Epoch 700 MSE = 0.540583
Epoch 800 MSE = 0.53607225
Epoch 900 MSE = 0.53281504


In [30]:
best_theta

array([[ 2.0685523 ],
       [ 0.7954762 ],
       [ 0.14298186],
       [-0.14372651],
       [ 0.17978257],
       [ 0.0047552 ],
       [-0.04064807],
       [-0.7553221 ],
       [-0.7192319 ]], dtype=float32)

## Using autodiff

In [33]:
def my_func(a, b): 
    z=0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [34]:
gradients = tf.gradients(mse, [theta])[0]

## Using an Optimizer

In [35]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [36]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)